### Data pre-processing: wps_climdexInput_csv

WPS wrapper for [climdexInput.csv](https://cran.r-project.org/web/packages/climdex.pcic/climdex.pcic.pdf#page=41) data pre-processing functions

In [1]:
import os
import requests
from birdy import WPSClient
from rpy2 import robjects
from urllib.request import urlretrieve
from pkg_resources import resource_filename
from tempfile import NamedTemporaryFile

from wps_tools.file_handling import csv_handler
from wps_tools.testing import get_target_url
from quail.utils import test_ci_output

In [2]:
# Ensure we are in the working directory with access to the data
while os.path.basename(os.getcwd()) != "quail":
    os.chdir('../')

In [3]:
# NBVAL_IGNORE_OUTPUT
url = get_target_url("quail")
print(f"Using quail on {url}")

Using quail on https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/quail/wps


In [4]:
quail = WPSClient(url)

#### Help for individual processes can be diplayed using the ? command (ex/ bird.process?)

In [5]:
# NBVAL_IGNORE_OUTPUT
quail.climdex_input_csv?

Signature:
quail.climdex_input_csv(
    prec_file_content,
    na_strings='NULL',
    tmax_column='tmax',
    tmin_column='tmin',
    prec_column='prec',
    tavg_column='tavg',
    base_range='c(1961, 1990)',
    cal='gregorian',
    date_fields="c('year', 'jday')",
    date_format='%Y %j',
    n=5,
    northern_hemisphere=True,
    quantiles='NULL',
    temp_qtiles='c(0.1, 0.9)',
    prec_qtiles='c(0.95, 0.99)',
    max_missing_days='c(annual = 15, monthly =3)',
    min_base_data_fraction_present=0.1,
    loglevel='INFO',
    tmax_file_content=None,
    tmin_file_content=None,
    tavg_file_content=None,
    output_file='output.rda',
    vector_name='output_vector',
    output_formats=None,
)
Docstring:
Process for creating climdexInput object from CSV files

Parameters
----------
tmax_file_content : string
    Content of file with daily maximum temperature data (temporary alternative to taking file).
tmin_file_content : string
    Content of file with daily minimum temperature data 

#### Run wps_climdexInput process

In [6]:
with NamedTemporaryFile(suffix=".rda", prefix="summer_days_", dir="/tmp", delete=True) as output_file:
    output = quail.climdex_input_csv(
            tmax_file_content=csv_handler(
                resource_filename("tests","data/1018935_MAX_TEMP.csv")),
            tmin_file_content=csv_handler(
                resource_filename("tests","data/1018935_MIN_TEMP.csv")),
            prec_file_content=csv_handler(
                resource_filename("tests","data/1018935_ONE_DAY_PRECIPITATION.csv")),
            tmax_column='MAX_TEMP',
            tmin_column='MIN_TEMP',
            prec_column='ONE_DAY_PRECIPITATION',
            base_range="c(1971, 2000)",
            vector_name="climdexInput",
        )
ci_url = output.get()[0]

In [7]:
test_ci_output(
        ci_url, "climdexInput", "climdexInput.rda", "ci"
    )